In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use(['seaborn'])
import seaborn as sns

# for loop status check
from tqdm import tqdm

import datetime

## data load

In [2]:
df_fin = pd.read_csv(r'./data/df_final.csv')
df_fin.drop(['Unnamed: 0'], axis=1, inplace=True)

time_list = ['order_purchase_timestamp',
             'order_approved_at',
             'order_delivered_carrier_date',
             'order_delivered_customer_date',
             'order_estimated_delivery_date'] 
for col in time_list:
    df_fin[col] = df_fin[col].apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    
df_fin['product_category_name'] = df_fin['product_category_name'].apply(lambda x: str(x))

In [3]:
df_data = df_fin[['order_id','seller_id','product_category_name','freight_value','distance_km','seller_work_diff','carrier_work_diff']]
# 이상치 제거 임의로 한거임
df_data = df_data[df_data['seller_work_diff']<30]
df_data = df_data[df_data['carrier_work_diff']<100]
df_seller = df_data[['order_id','seller_id','product_category_name','seller_work_diff']]
df_carrier = df_data[['order_id','product_category_name','freight_value','distance_km','carrier_work_diff']]

In [15]:
df_fin['predict_work_diff']

0         11.583333
1         13.166667
2         18.666667
3         18.208333
4          8.125000
            ...    
109922    17.500000
109923    21.416667
109924    27.125000
109925    27.125000
109926    17.166667
Name: predict_work_diff, Length: 109927, dtype: float64

## olist 예측

In [48]:
from sklearn.model_selection import train_test_split

x = df_seller[['seller_id','product_category_name']]
y = df_seller['seller_work_diff']

# 테스트 데이터 30%
x_train, x_test, y_train, y_seller_test = train_test_split(x, y, test_size=0.2, random_state=42)

87878
21970
87878
21970


In [49]:
x = df_carrier[['product_category_name','distance_km','freight_value']]
y = df_carrier['carrier_work_diff']

from sklearn.preprocessing import LabelEncoder
# 라벨 인코더 생성

encoder = LabelEncoder()
encoder.fit(x['product_category_name'])

x['product_category_name'] = encoder.transform(x['product_category_name'])
#     # X_train데이터를 이용 피팅하고 

from sklearn.model_selection import train_test_split
# 테스트 데이터 30%
x_train, x_test, y_train, y_carrier_test = train_test_split(x_encoded, y, test_size=0.2, random_state=42)

<ipython-input-49-9f621d17fc1f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['product_category_name'] = encoder.transform(x['product_category_name'])


In [54]:
y_predict_work_diff = df_fin['predict_work_diff'][y_test.index]
print('Total Carrier RMSE :', metrics.mean_squared_error(y_seller_test+y_carrier_test, y_predict_work_diff))

Total Carrier RMSE : 108.79074536179775


## CatBoost

In [57]:
from catboost import CatBoostRegressor, FeaturesData, Pool

<br>**Seller spent time**

In [58]:

from sklearn.model_selection import train_test_split

x = df_seller[['seller_id','product_category_name']]
y = df_seller['seller_work_diff']

# 테스트 데이터 20%
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

87878
21970
87878
21970


In [59]:
train_pool = Pool(
    data = FeaturesData(cat_feature_data = np.array(x_train.values, dtype=object), 
                    cat_feature_names = list(x_train.columns.values)),
    label =  np.array(y_train, dtype=np.float32)
)

In [60]:
test_pool = Pool(
    data = FeaturesData(cat_feature_data = np.array(x_test.values, dtype=object), 
                    cat_feature_names = list(x_test.columns.values))
)

In [61]:
model = CatBoostRegressor(iterations=1000, learning_rate=0.05, depth=7)
# Fit model
model.fit(train_pool)
# Get predictions
preds = model.predict(test_pool)

0:	learn: 2.3665556	total: 91ms	remaining: 1m 30s
1:	learn: 2.3453363	total: 119ms	remaining: 59.4s
2:	learn: 2.3246954	total: 147ms	remaining: 48.9s
3:	learn: 2.3060548	total: 172ms	remaining: 42.9s
4:	learn: 2.2887782	total: 201ms	remaining: 40s
5:	learn: 2.2729488	total: 229ms	remaining: 37.9s
6:	learn: 2.2591711	total: 255ms	remaining: 36.2s
7:	learn: 2.2460627	total: 280ms	remaining: 34.7s
8:	learn: 2.2357104	total: 306ms	remaining: 33.7s
9:	learn: 2.2246533	total: 338ms	remaining: 33.4s
10:	learn: 2.2143902	total: 361ms	remaining: 32.4s
11:	learn: 2.2052347	total: 389ms	remaining: 32s
12:	learn: 2.1977213	total: 420ms	remaining: 31.9s
13:	learn: 2.1906798	total: 446ms	remaining: 31.4s
14:	learn: 2.1837058	total: 475ms	remaining: 31.2s
15:	learn: 2.1780060	total: 507ms	remaining: 31.2s
16:	learn: 2.1719533	total: 536ms	remaining: 31s
17:	learn: 2.1667651	total: 563ms	remaining: 30.7s
18:	learn: 2.1617565	total: 587ms	remaining: 30.3s
19:	learn: 2.1571230	total: 610ms	remaining: 29

167:	learn: 2.0713409	total: 4.61s	remaining: 22.8s
168:	learn: 2.0712837	total: 4.64s	remaining: 22.8s
169:	learn: 2.0712830	total: 4.65s	remaining: 22.7s
170:	learn: 2.0711078	total: 4.68s	remaining: 22.7s
171:	learn: 2.0710824	total: 4.71s	remaining: 22.7s
172:	learn: 2.0710672	total: 4.74s	remaining: 22.6s
173:	learn: 2.0706257	total: 4.77s	remaining: 22.6s
174:	learn: 2.0705336	total: 4.8s	remaining: 22.6s
175:	learn: 2.0703927	total: 4.83s	remaining: 22.6s
176:	learn: 2.0703631	total: 4.85s	remaining: 22.6s
177:	learn: 2.0701994	total: 4.88s	remaining: 22.5s
178:	learn: 2.0701135	total: 4.91s	remaining: 22.5s
179:	learn: 2.0700684	total: 4.94s	remaining: 22.5s
180:	learn: 2.0700044	total: 4.96s	remaining: 22.5s
181:	learn: 2.0698605	total: 4.99s	remaining: 22.4s
182:	learn: 2.0698305	total: 5.02s	remaining: 22.4s
183:	learn: 2.0698095	total: 5.05s	remaining: 22.4s
184:	learn: 2.0697585	total: 5.07s	remaining: 22.4s
185:	learn: 2.0696476	total: 5.1s	remaining: 22.3s
186:	learn: 2.

332:	learn: 2.0550886	total: 8.64s	remaining: 17.3s
333:	learn: 2.0550573	total: 8.67s	remaining: 17.3s
334:	learn: 2.0550070	total: 8.69s	remaining: 17.2s
335:	learn: 2.0549827	total: 8.71s	remaining: 17.2s
336:	learn: 2.0548968	total: 8.73s	remaining: 17.2s
337:	learn: 2.0548826	total: 8.76s	remaining: 17.1s
338:	learn: 2.0546812	total: 8.79s	remaining: 17.1s
339:	learn: 2.0545915	total: 8.81s	remaining: 17.1s
340:	learn: 2.0545476	total: 8.84s	remaining: 17.1s
341:	learn: 2.0543917	total: 8.87s	remaining: 17.1s
342:	learn: 2.0542738	total: 8.9s	remaining: 17s
343:	learn: 2.0541689	total: 8.92s	remaining: 17s
344:	learn: 2.0541595	total: 8.95s	remaining: 17s
345:	learn: 2.0541121	total: 8.98s	remaining: 17s
346:	learn: 2.0540513	total: 9s	remaining: 16.9s
347:	learn: 2.0540334	total: 9.02s	remaining: 16.9s
348:	learn: 2.0538745	total: 9.04s	remaining: 16.9s
349:	learn: 2.0538623	total: 9.06s	remaining: 16.8s
350:	learn: 2.0536656	total: 9.08s	remaining: 16.8s
351:	learn: 2.0536301	to

497:	learn: 2.0463784	total: 12.8s	remaining: 12.9s
498:	learn: 2.0463571	total: 12.8s	remaining: 12.8s
499:	learn: 2.0463152	total: 12.8s	remaining: 12.8s
500:	learn: 2.0462933	total: 12.8s	remaining: 12.8s
501:	learn: 2.0462709	total: 12.8s	remaining: 12.7s
502:	learn: 2.0462402	total: 12.9s	remaining: 12.7s
503:	learn: 2.0462247	total: 12.9s	remaining: 12.7s
504:	learn: 2.0461967	total: 12.9s	remaining: 12.7s
505:	learn: 2.0461146	total: 12.9s	remaining: 12.6s
506:	learn: 2.0461014	total: 13s	remaining: 12.6s
507:	learn: 2.0460765	total: 13s	remaining: 12.6s
508:	learn: 2.0460338	total: 13s	remaining: 12.5s
509:	learn: 2.0460257	total: 13s	remaining: 12.5s
510:	learn: 2.0459573	total: 13s	remaining: 12.5s
511:	learn: 2.0458300	total: 13.1s	remaining: 12.5s
512:	learn: 2.0457576	total: 13.1s	remaining: 12.4s
513:	learn: 2.0457332	total: 13.1s	remaining: 12.4s
514:	learn: 2.0457021	total: 13.1s	remaining: 12.4s
515:	learn: 2.0456117	total: 13.2s	remaining: 12.3s
516:	learn: 2.0455452	

663:	learn: 2.0393063	total: 17s	remaining: 8.59s
664:	learn: 2.0392925	total: 17s	remaining: 8.57s
665:	learn: 2.0392198	total: 17s	remaining: 8.54s
666:	learn: 2.0392191	total: 17.1s	remaining: 8.51s
667:	learn: 2.0391830	total: 17.1s	remaining: 8.48s
668:	learn: 2.0391654	total: 17.1s	remaining: 8.46s
669:	learn: 2.0390859	total: 17.1s	remaining: 8.43s
670:	learn: 2.0390797	total: 17.1s	remaining: 8.4s
671:	learn: 2.0390540	total: 17.2s	remaining: 8.38s
672:	learn: 2.0390442	total: 17.2s	remaining: 8.35s
673:	learn: 2.0390435	total: 17.2s	remaining: 8.32s
674:	learn: 2.0390337	total: 17.2s	remaining: 8.29s
675:	learn: 2.0389770	total: 17.3s	remaining: 8.27s
676:	learn: 2.0389688	total: 17.3s	remaining: 8.24s
677:	learn: 2.0389502	total: 17.3s	remaining: 8.21s
678:	learn: 2.0389405	total: 17.3s	remaining: 8.19s
679:	learn: 2.0388876	total: 17.3s	remaining: 8.16s
680:	learn: 2.0388325	total: 17.4s	remaining: 8.13s
681:	learn: 2.0388138	total: 17.4s	remaining: 8.11s
682:	learn: 2.03878

828:	learn: 2.0339510	total: 21.2s	remaining: 4.38s
829:	learn: 2.0339320	total: 21.3s	remaining: 4.36s
830:	learn: 2.0339230	total: 21.3s	remaining: 4.33s
831:	learn: 2.0338891	total: 21.3s	remaining: 4.3s
832:	learn: 2.0338524	total: 21.3s	remaining: 4.28s
833:	learn: 2.0337911	total: 21.4s	remaining: 4.25s
834:	learn: 2.0337702	total: 21.4s	remaining: 4.23s
835:	learn: 2.0337510	total: 21.4s	remaining: 4.2s
836:	learn: 2.0337456	total: 21.4s	remaining: 4.18s
837:	learn: 2.0337085	total: 21.5s	remaining: 4.15s
838:	learn: 2.0337015	total: 21.5s	remaining: 4.13s
839:	learn: 2.0336642	total: 21.5s	remaining: 4.1s
840:	learn: 2.0336391	total: 21.6s	remaining: 4.08s
841:	learn: 2.0336175	total: 21.6s	remaining: 4.05s
842:	learn: 2.0336103	total: 21.6s	remaining: 4.03s
843:	learn: 2.0335991	total: 21.6s	remaining: 4s
844:	learn: 2.0335752	total: 21.7s	remaining: 3.98s
845:	learn: 2.0335468	total: 21.7s	remaining: 3.95s
846:	learn: 2.0335276	total: 21.7s	remaining: 3.92s
847:	learn: 2.0334

991:	learn: 2.0298333	total: 25.3s	remaining: 204ms
992:	learn: 2.0298200	total: 25.4s	remaining: 179ms
993:	learn: 2.0297834	total: 25.4s	remaining: 153ms
994:	learn: 2.0297776	total: 25.4s	remaining: 128ms
995:	learn: 2.0297365	total: 25.4s	remaining: 102ms
996:	learn: 2.0297019	total: 25.4s	remaining: 76.6ms
997:	learn: 2.0296914	total: 25.5s	remaining: 51ms
998:	learn: 2.0296782	total: 25.5s	remaining: 25.5ms
999:	learn: 2.0296657	total: 25.5s	remaining: 0us


In [62]:
# 정확도 확인
seller_rmse =  metrics.mean_squared_error(y_test, preds)
# print('seller_rmse :', seller_rmse)

<br>**carrier spent time**

In [63]:
from sklearn.model_selection import train_test_split

x = df_carrier[['product_category_name','distance_km','freight_value']]
y = df_carrier['carrier_work_diff']

# 테스트 데이터 30%
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

87878
21970
87878
21970


In [65]:
np.array(x_train[['distance_km','freight_value']], dtype=np.float32).shape

(87878, 2)

In [66]:
np.array(x_train['product_category_name']).reshape(-1,1)

array([['beleza_saude'],
       ['automotivo'],
       ['telefonia'],
       ...,
       ['moveis_decoracao'],
       ['telefonia'],
       ['informatica_acessorios']], dtype=object)

In [67]:
train_pool = Pool(
    data = FeaturesData(
        num_feature_data = np.array(x_train[['distance_km','freight_value']], dtype=np.float32), 
        cat_feature_data = np.array(x_train['product_category_name'], dtype=object).reshape(-1,1), 
        num_feature_names = ['distance_km','freight_value'], 
        cat_feature_names = ['product_category_name']
    ),
    label =  np.array(y_train, dtype=np.float32)
)

In [68]:
test_pool = Pool(
    data = FeaturesData(num_feature_data = np.array(x_test[['distance_km','freight_value']].values, dtype=np.float32), 
                    cat_feature_data = np.array(x_test['product_category_name'], dtype=object).reshape(-1,1),  
                    num_feature_names = ['distance_km','freight_value'], 
                    cat_feature_names = ['product_category_name'])
)

In [69]:
model = CatBoostRegressor(iterations=1000, learning_rate=0.05, depth=7)
# Fit model
model.fit(train_pool)
# Get predictions
preds = model.predict(test_pool)

0:	learn: 5.7921765	total: 36.8ms	remaining: 36.8s
1:	learn: 5.7325539	total: 75.9ms	remaining: 37.9s
2:	learn: 5.6784847	total: 115ms	remaining: 38.2s
3:	learn: 5.6297756	total: 150ms	remaining: 37.3s
4:	learn: 5.5845893	total: 193ms	remaining: 38.4s
5:	learn: 5.5432673	total: 228ms	remaining: 37.7s
6:	learn: 5.5061956	total: 264ms	remaining: 37.5s
7:	learn: 5.4721905	total: 298ms	remaining: 37s
8:	learn: 5.4412636	total: 332ms	remaining: 36.5s
9:	learn: 5.4133126	total: 364ms	remaining: 36s
10:	learn: 5.3879159	total: 415ms	remaining: 37.3s
11:	learn: 5.3647891	total: 459ms	remaining: 37.8s
12:	learn: 5.3433722	total: 503ms	remaining: 38.2s
13:	learn: 5.3237706	total: 556ms	remaining: 39.1s
14:	learn: 5.3058500	total: 598ms	remaining: 39.3s
15:	learn: 5.2900283	total: 642ms	remaining: 39.5s
16:	learn: 5.2757646	total: 680ms	remaining: 39.3s
17:	learn: 5.2623500	total: 721ms	remaining: 39.3s
18:	learn: 5.2504785	total: 767ms	remaining: 39.6s
19:	learn: 5.2393990	total: 818ms	remaining

165:	learn: 5.0896144	total: 5.24s	remaining: 26.3s
166:	learn: 5.0893259	total: 5.27s	remaining: 26.3s
167:	learn: 5.0891415	total: 5.3s	remaining: 26.3s
168:	learn: 5.0888154	total: 5.33s	remaining: 26.2s
169:	learn: 5.0887367	total: 5.36s	remaining: 26.2s
170:	learn: 5.0885587	total: 5.38s	remaining: 26.1s
171:	learn: 5.0884773	total: 5.41s	remaining: 26s
172:	learn: 5.0884147	total: 5.43s	remaining: 26s
173:	learn: 5.0883319	total: 5.46s	remaining: 25.9s
174:	learn: 5.0883199	total: 5.47s	remaining: 25.8s
175:	learn: 5.0881505	total: 5.5s	remaining: 25.7s
176:	learn: 5.0878809	total: 5.53s	remaining: 25.7s
177:	learn: 5.0878025	total: 5.55s	remaining: 25.6s
178:	learn: 5.0874407	total: 5.58s	remaining: 25.6s
179:	learn: 5.0874064	total: 5.6s	remaining: 25.5s
180:	learn: 5.0873279	total: 5.63s	remaining: 25.5s
181:	learn: 5.0870995	total: 5.66s	remaining: 25.4s
182:	learn: 5.0866136	total: 5.68s	remaining: 25.4s
183:	learn: 5.0863362	total: 5.71s	remaining: 25.3s
184:	learn: 5.08623

326:	learn: 5.0551020	total: 10.3s	remaining: 21.1s
327:	learn: 5.0545720	total: 10.3s	remaining: 21.1s
328:	learn: 5.0543537	total: 10.3s	remaining: 21s
329:	learn: 5.0540373	total: 10.3s	remaining: 21s
330:	learn: 5.0537131	total: 10.4s	remaining: 21s
331:	learn: 5.0534927	total: 10.4s	remaining: 20.9s
332:	learn: 5.0532209	total: 10.4s	remaining: 20.9s
333:	learn: 5.0528891	total: 10.5s	remaining: 20.9s
334:	learn: 5.0527437	total: 10.5s	remaining: 20.8s
335:	learn: 5.0525413	total: 10.5s	remaining: 20.8s
336:	learn: 5.0525018	total: 10.6s	remaining: 20.8s
337:	learn: 5.0522354	total: 10.6s	remaining: 20.7s
338:	learn: 5.0520686	total: 10.6s	remaining: 20.7s
339:	learn: 5.0520111	total: 10.6s	remaining: 20.7s
340:	learn: 5.0518128	total: 10.7s	remaining: 20.6s
341:	learn: 5.0516366	total: 10.7s	remaining: 20.6s
342:	learn: 5.0513402	total: 10.7s	remaining: 20.5s
343:	learn: 5.0510887	total: 10.8s	remaining: 20.5s
344:	learn: 5.0508159	total: 10.8s	remaining: 20.5s
345:	learn: 5.0506

486:	learn: 5.0263149	total: 15.2s	remaining: 16s
487:	learn: 5.0262250	total: 15.3s	remaining: 16s
488:	learn: 5.0260624	total: 15.3s	remaining: 16s
489:	learn: 5.0260346	total: 15.3s	remaining: 15.9s
490:	learn: 5.0259512	total: 15.3s	remaining: 15.9s
491:	learn: 5.0257142	total: 15.4s	remaining: 15.9s
492:	learn: 5.0256397	total: 15.4s	remaining: 15.8s
493:	learn: 5.0253864	total: 15.4s	remaining: 15.8s
494:	learn: 5.0250957	total: 15.5s	remaining: 15.8s
495:	learn: 5.0250662	total: 15.5s	remaining: 15.7s
496:	learn: 5.0248396	total: 15.5s	remaining: 15.7s
497:	learn: 5.0246511	total: 15.5s	remaining: 15.7s
498:	learn: 5.0244257	total: 15.6s	remaining: 15.6s
499:	learn: 5.0242344	total: 15.6s	remaining: 15.6s
500:	learn: 5.0240483	total: 15.6s	remaining: 15.6s
501:	learn: 5.0238051	total: 15.6s	remaining: 15.5s
502:	learn: 5.0236462	total: 15.7s	remaining: 15.5s
503:	learn: 5.0234393	total: 15.7s	remaining: 15.5s
504:	learn: 5.0233903	total: 15.7s	remaining: 15.4s
505:	learn: 5.0232

650:	learn: 5.0029925	total: 20.3s	remaining: 10.9s
651:	learn: 5.0028660	total: 20.3s	remaining: 10.8s
652:	learn: 5.0027752	total: 20.3s	remaining: 10.8s
653:	learn: 5.0026579	total: 20.4s	remaining: 10.8s
654:	learn: 5.0026068	total: 20.4s	remaining: 10.8s
655:	learn: 5.0023772	total: 20.5s	remaining: 10.7s
656:	learn: 5.0022393	total: 20.5s	remaining: 10.7s
657:	learn: 5.0020911	total: 20.5s	remaining: 10.7s
658:	learn: 5.0019917	total: 20.5s	remaining: 10.6s
659:	learn: 5.0019445	total: 20.6s	remaining: 10.6s
660:	learn: 5.0019004	total: 20.6s	remaining: 10.6s
661:	learn: 5.0018533	total: 20.6s	remaining: 10.5s
662:	learn: 5.0017640	total: 20.7s	remaining: 10.5s
663:	learn: 5.0016443	total: 20.7s	remaining: 10.5s
664:	learn: 5.0015635	total: 20.7s	remaining: 10.4s
665:	learn: 5.0014156	total: 20.8s	remaining: 10.4s
666:	learn: 5.0013484	total: 20.8s	remaining: 10.4s
667:	learn: 5.0011328	total: 20.8s	remaining: 10.3s
668:	learn: 5.0010660	total: 20.8s	remaining: 10.3s
669:	learn: 

812:	learn: 4.9835324	total: 25.3s	remaining: 5.83s
813:	learn: 4.9834607	total: 25.4s	remaining: 5.8s
814:	learn: 4.9834434	total: 25.4s	remaining: 5.76s
815:	learn: 4.9833155	total: 25.4s	remaining: 5.73s
816:	learn: 4.9832044	total: 25.5s	remaining: 5.7s
817:	learn: 4.9831395	total: 25.5s	remaining: 5.67s
818:	learn: 4.9829781	total: 25.5s	remaining: 5.64s
819:	learn: 4.9829084	total: 25.6s	remaining: 5.61s
820:	learn: 4.9826683	total: 25.6s	remaining: 5.58s
821:	learn: 4.9825515	total: 25.6s	remaining: 5.55s
822:	learn: 4.9824625	total: 25.7s	remaining: 5.52s
823:	learn: 4.9823503	total: 25.7s	remaining: 5.49s
824:	learn: 4.9821983	total: 25.7s	remaining: 5.46s
825:	learn: 4.9820393	total: 25.8s	remaining: 5.43s
826:	learn: 4.9818974	total: 25.8s	remaining: 5.4s
827:	learn: 4.9818806	total: 25.8s	remaining: 5.37s
828:	learn: 4.9816857	total: 25.9s	remaining: 5.33s
829:	learn: 4.9815723	total: 25.9s	remaining: 5.3s
830:	learn: 4.9814972	total: 25.9s	remaining: 5.27s
831:	learn: 4.98

971:	learn: 4.9662281	total: 30.3s	remaining: 874ms
972:	learn: 4.9661212	total: 30.4s	remaining: 843ms
973:	learn: 4.9661048	total: 30.4s	remaining: 812ms
974:	learn: 4.9660588	total: 30.4s	remaining: 781ms
975:	learn: 4.9657482	total: 30.5s	remaining: 749ms
976:	learn: 4.9656370	total: 30.5s	remaining: 718ms
977:	learn: 4.9653968	total: 30.6s	remaining: 688ms
978:	learn: 4.9653204	total: 30.6s	remaining: 656ms
979:	learn: 4.9651271	total: 30.6s	remaining: 625ms
980:	learn: 4.9650931	total: 30.7s	remaining: 594ms
981:	learn: 4.9649173	total: 30.7s	remaining: 562ms
982:	learn: 4.9648203	total: 30.7s	remaining: 531ms
983:	learn: 4.9647359	total: 30.7s	remaining: 500ms
984:	learn: 4.9646461	total: 30.8s	remaining: 469ms
985:	learn: 4.9645881	total: 30.8s	remaining: 438ms
986:	learn: 4.9644738	total: 30.8s	remaining: 406ms
987:	learn: 4.9644615	total: 30.9s	remaining: 375ms
988:	learn: 4.9643377	total: 30.9s	remaining: 344ms
989:	learn: 4.9643258	total: 30.9s	remaining: 313ms
990:	learn: 

In [70]:
# 정확도 확인
carrier_rmse = metrics.mean_squared_error(y_test, preds)
print('seller_rmse :', seller_rmse)
print('carrier_rmse :', carrier_rmse)
print('carrier_rmse :', seller_rmse+carrier_rmse)


seller_rmse : 4.289891629434166
carrier_rmse : 24.06089805064122
carrier_rmse : 28.350789680075387
